In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
gzdekzlkaya_wikipedia_text_corpus_for_nlp_and_llm_projects_path = kagglehub.dataset_download('gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects')

print('Data source import complete.')


100%|██████████| 18.7M/18.7M [00:00<00:00, 86.4MB/s]

Extracting files...


Data source import complete.


# Ejercicio 7: Bases de Datos Vectoriales

## Objetivo de la práctica

Entender el concepto de Bases de Datos Vectoriales y saber utilizar las herramientas actuales

## Parte 0: Carga del Corpus

Vamos a utilizar la API de Kaggle para acceder al dataset _Wikipedia Text Corpus for NLP and LLM Projects_

El corpus está disponible desde este [link](https://www.kaggle.com/datasets/gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects?utm_source=chatgpt.com)

### Actividad

1. Carga el corpus


In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [ ]:
# Set the path to the file you'd like to load
file_path = "wikipedia_text_corpus.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects",
  file_path,
)

df.head()

Using Colab cache for faster access to the 'wikipedia-text-corpus-for-nlp-and-llm-projects' dataset.


,Unnamed: 0,text
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...
1,2,Battery indicator\n\nA battery indicator (also...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...


## Parte 1: Generación de Embeddings

Vamos a utilizar E5 como modelo de embeddings.

La documentación de E5 está disponible desde este [link](https://huggingface.co/intfloat/e5-base-v2)

### Actividad

1. Normalizar el corpus
2. Definir una función `chunk_text`, y dividir los textos en _chunks_.
3. Generar embeddings por cada _chunk_

In [ ]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

df = df.dropna(subset=["text"]).reset_index(drop=True)

# Limpieza básica
def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s


df["text_norm"] = df["text"].astype(str).map(normalize_text)

df.head()

,Unnamed: 0,text,text_norm
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...,Anovo Anovo (formerly A Novo) is a computer se...
1,2,Battery indicator\n\nA battery indicator (also...,Battery indicator A battery indicator (also kn...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19...","Bob Pease Robert Allen Pease (August 22, 1940Â..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...,CAVNET CAVNET was a secure military forum whic...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...,CLidar The CLidar is a scientific instrument u...


In [ ]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    """
    Chunking por caracteres.
    max_chars ~ 600-1000 suele funcionar bien.
    overlap ayuda a no cortar ideas a la mitad.
    """
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

(   doc_id  chunk_id                                               text
 0       0         0  Anovo Anovo (formerly A Novo) is a computer se...
 1       1         0  Battery indicator A battery indicator (also kn...
 2       1         1  ad battery when in reality it indicates a prob...
 3       1         2  s that an internal standby battery needs repla...
 4       1         3  increase; in many cases the EMF remains more o...,
 79104)

In [ ]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
model = SentenceTransformer(MODEL_NAME)

# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [ ]:
# Embeddings (N x D)
# Se debe usar normalize_embeddings=True para similitud coseno
embeddings = model.encode(
    passages,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

Batches:   0%|          | 0/4944 [00:00<?, ?it/s]

In [ ]:
print(embeddings.shape, embeddings.dtype)

(79104, 768) float32


In [ ]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Battery measuring"

query_vec = embed_query(query_text)
query_vec.shape

(1, 768)

## Parte 2: FAISS

FAISS es una librería para búsqueda por similitud eficiente y clustering de vectores densos.

La documentación de FAISS está disponible en este [link](https://faiss.ai/index.html)

### Actividad

1. Crea un índice en FAISS
2. Carga los embeddings
3. Realiza una búsqueda a partir de una _query_

In [ ]:
!pip install -q faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 96.9 MB/s eta 0:00:00


In [ ]:
# código base para FAISS
import faiss
import numpy as np

# Asumiendo `embeddings` en un array NxD
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

D, I = index.search(query_vec, k=10)


In [ ]:
for rank, (idx, score) in enumerate(zip(I[0], D[0]), 1):
    print(f"#{rank} | score: {score:.4f}")
    print(passages[idx][:200])
    print("-" * 50)


#1 | score: 0.2593
passage: Battery tester A battery tester is an electronic device intended for testing the state of an electric battery, going from a simple device for testing the charge actually present in the cells 
--------------------------------------------------
#2 | score: 0.2764
passage: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of c
--------------------------------------------------
#3 | score: 0.3198
passage: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-acid vehicle batteries. Their common principle is based on the empirical fact that after havi
--------------------------------------------------
#4 | score: 0.3217
passage: ils. One was connected via a series resistor to the battery supply. The second was connected to the same battery supply via a second resistor and the resistor 

## Parte 3 — Vector DB #1: Qdrant (búsqueda vectorial + metadata)

### Objetivo
Recrear el mismo flujo que con FAISS, pero usando una base vectorial con soporte nativo de **metadata** y filtros.

### Qué debes implementar
1. Levantar / conectar con una instancia de Qdrant.
2. Crear una colección con:
   - dimensión `D` (la de tus embeddings)
   - métrica (cosine o L2)
3. Insertar:
   - `id`
   - `embedding`
   - `payload` (metadata: texto, título, etiquetas, etc.)
4. Consultar Top-k por similitud:
   - `query_embedding`
   - `k`

### Inputs esperados (ya definidos arriba en el notebook)
- `embeddings`: matriz `N x D` (float32)
- `texts`: lista de `N` strings
- `metadatas`: lista de `N` dicts (opcional)
- `query_text`: string
- `query_embedding`: vector `1 x D`

### Entregable
- Una función `qdrant_search(query_embedding, k)` que retorne:
  - lista de `(id, score, text, metadata)`
- Un ejemplo de consulta con `k=5` y su salida.

### Preguntas
- ¿La métrica usada fue cosine o L2? ¿Por qué?
- ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?
- ¿Qué pasa con el tiempo de respuesta cuando aumentas `k`?


In [ ]:
!pip install -q qdrant-client


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.2/377.2 kB 14.9 MB/s eta 0:00:00


In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct


In [ ]:
client = QdrantClient(location=":memory:")


In [ ]:
DIM = embeddings.shape[1]

client.recreate_collection(
    collection_name="documents",
    vectors_config=VectorParams(
        size=DIM,
        distance=Distance.COSINE
    )
)


/tmp/ipython-input-2360850943.py:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [ ]:
texts = passages


In [ ]:
points = [
    PointStruct(
        id=i,
        vector=embeddings[i].tolist(),
        payload={"text": texts[i]}
    )
    for i in range(len(texts))
]

client.upsert(
    collection_name="documents",
    points=points
)


/tmp/ipython-input-4202245425.py:10: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Current collection contains 79104 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  client.upsert(


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
def qdrant_search(query_embedding, k=5):
    results = client.query_points(
        collection_name="documents",
        prefetch=[],
        query=query_embedding[0].tolist(),
        limit=k,
        with_payload=True
    )

    output = []
    for r in results.points:
        output.append((
            r.id,
            r.score,
            r.payload["text"],
            {k: v for k, v in r.payload.items() if k != "text"}
        ))
    return output


In [ ]:
results = qdrant_search(query_vec, k=5)

for r in results:
    print(f"ID: {r[0]} | Score: {r[1]:.4f}")
    print(r[2][:200])
    print("Metadata:", r[3])
    print("-" * 50)


ID: 10176 | Score: 0.8703
passage: Battery tester A battery tester is an electronic device intended for testing the state of an electric battery, going from a simple device for testing the charge actually present in the cells 
Metadata: {}
--------------------------------------------------
ID: 1 | Score: 0.8618
passage: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of c
Metadata: {}
--------------------------------------------------
ID: 10177 | Score: 0.8401
passage: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-acid vehicle batteries. Their common principle is based on the empirical fact that after havi
Metadata: {}
--------------------------------------------------
ID: 37406 | Score: 0.8391
passage: ils. One was connected via a series resistor to the battery supply. The second was connected to 


RES
*  Se utilizó similitud coseno, ya que los embeddings fueron previamente normalizados. La similitud coseno mide de forma efectiva la cercanía semántica entre los textos, independientemente de la magnitud de los vectores. Elemento de la lista
*  En Qdrant, el filtrado por metadata es nativo y sencillo, ya que cada vector puede almacenarse junto con un payload que permite aplicar filtros directamente durante la consulta. En contraste, FAISS no soporta metadata de forma nativa, por lo que cualquier filtrado debe implementarse manualmente fuera del índice vectorial, lo que hace el proceso menos flexible.
* Al aumentar el valor de k, el tiempo de respuesta incrementa ligeramente, ya que el motor debe recuperar y ordenar un mayor número de vectores similares.

## Parte 4 — Vector DB #2: Milvus (indexación ANN y escalabilidad)

### Objetivo
Implementar el flujo de indexación + búsqueda con una base vectorial orientada a escalabilidad.

### Qué debes implementar
1. Conectar a Milvus.
2. Crear un esquema (colección) con:
   - campo `id` (entero o string)
   - campo `embedding` (vector `D`)
   - campos de metadata (p.ej., `category`, `source`, `title`)
3. Insertar `N` embeddings.
4. Crear/seleccionar un índice ANN (ej. HNSW o IVF).
5. Ejecutar consultas Top-k y recuperar textos asociados.

### Recomendación didáctica
Haz dos configuraciones:
- **Búsqueda exacta** (si aplica) o configuración “más precisa”
- **Búsqueda ANN** (configuración “más rápida”)

Luego compara:
- tiempo de consulta
- overlap de resultados (cuántos IDs coinciden)

### Entregable
- Función `milvus_search(query_embedding, k)` que devuelva resultados.
- Un mini experimento: `k=5` y `k=20` (tiempos y resultados).

### Preguntas
- ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?
- ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?


In [ ]:
!pip install -q "pymilvus[milvus_lite]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 17.9 MB/s eta 0:00:00


In [ ]:
from pymilvus import connections

connections.connect(
    alias="default",
    uri="milvus_demo.db"
)


In [ ]:
from pymilvus import utility

if utility.has_collection("documents_milvus"):
    utility.drop_collection("documents_milvus")
    print("Colección eliminada")


In [ ]:
from pymilvus import (
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection
)

DIM = embeddings.shape[1]

fields = [
    FieldSchema(
        name="id",
        dtype=DataType.INT64,
        is_primary=True,
        auto_id=False
    ),
    FieldSchema(
        name="embedding",
        dtype=DataType.FLOAT_VECTOR,
        dim=DIM
    ),
    FieldSchema(
        name="text",
        dtype=DataType.VARCHAR,
        max_length=2000
    )
]

schema = CollectionSchema(
    fields=fields,
    description="Wikipedia passages"
)

collection = Collection(
    name="documents_milvus",
    schema=schema
)

print("Colección creada")


Colección creada


In [ ]:
BATCH_SIZE = 2000

for start in range(0, len(texts), BATCH_SIZE):
    end = start + BATCH_SIZE

    batch_ids = list(range(start, min(end, len(texts))))
    batch_vectors = embeddings[start:end].tolist()
    batch_texts = texts[start:end]

    collection.insert([
        batch_ids,
        batch_vectors,
        batch_texts
    ])

    print(f"Insertados {end} / {len(texts)}")


Insertados 2000 / 79104
Insertados 4000 / 79104
Insertados 6000 / 79104
Insertados 8000 / 79104
Insertados 10000 / 79104
Insertados 12000 / 79104
Insertados 14000 / 79104
Insertados 16000 / 79104
Insertados 18000 / 79104
Insertados 20000 / 79104
Insertados 22000 / 79104
Insertados 24000 / 79104
Insertados 26000 / 79104
Insertados 28000 / 79104
Insertados 30000 / 79104
Insertados 32000 / 79104
Insertados 34000 / 79104
Insertados 36000 / 79104
Insertados 38000 / 79104
Insertados 40000 / 79104
Insertados 42000 / 79104
Insertados 44000 / 79104
Insertados 46000 / 79104
Insertados 48000 / 79104
Insertados 50000 / 79104
Insertados 52000 / 79104
Insertados 54000 / 79104
Insertados 56000 / 79104
Insertados 58000 / 79104
Insertados 60000 / 79104
Insertados 62000 / 79104
Insertados 64000 / 79104
Insertados 66000 / 79104
Insertados 68000 / 79104
Insertados 70000 / 79104
Insertados 72000 / 79104
Insertados 74000 / 79104
Insertados 76000 / 79104
Insertados 78000 / 79104
Insertados 80000 / 79104


In [ ]:
collection.flush()
print("Total entidades:", collection.num_entities)


Total entidades: 79104


In [ ]:
# Crear indice
# Borrar índices existentes
for idx in collection.indexes:
    collection.drop_index(index_name=idx.index_name)

# Índice exacto
index_flat = {
    "index_type": "FLAT",
    "metric_type": "COSINE",
    "params": {}
}

collection.create_index(
    field_name="embedding",
    index_params=index_flat
)



Status(code=0, message=)

In [ ]:
# Borrar índice FLAT
for idx in collection.indexes:
    collection.drop_index(index_name=idx.index_name)

index_ivf = {
    "index_type": "IVF_FLAT",
    "metric_type": "COSINE",
    "params": {
        "nlist": 128
    }
}

collection.create_index(
    field_name="embedding",
    index_params=index_ivf
)


Status(code=0, message=)

In [ ]:
collection.load()

In [ ]:
import time

def milvus_search(query_embedding, k=5, nprobe=16):
    search_params = {
        "metric_type": "COSINE",
        "params": {
            "nprobe": nprobe
        }
    }

    start = time.time()
    results = collection.search(
        data=[query_embedding[0].tolist()],
        anns_field="embedding",
        param=search_params,
        limit=k,
        output_fields=["text"]
    )
    elapsed = time.time() - start

    output = []
    for hit in results[0]:
        output.append(
            (hit.id, hit.score, hit.entity.get("text"))
        )

    return output, elapsed


In [ ]:
res_5, t5 = milvus_search(query_vec, k=5)
res_20, t20 = milvus_search(query_vec, k=20)

print("k=5  | tiempo:", t5)
print("k=20 | tiempo:", t20)


k=5  | tiempo: 0.04378771781921387
k=20 | tiempo: 0.03890252113342285


In [ ]:
ids_5 = set(r[0] for r in res_5)
ids_20 = set(r[0] for r in res_20)

print("Overlap:", len(ids_5 & ids_20), "/ 5")


Overlap: 5 / 5




*   Se utilizó un índice FLAT para búsqueda exacta y un índice IVF_FLAT para búsqueda aproximada. En IVF_FLAT se ajustó el parámetro nlist, que controla la cantidad de clusters, y nprobe durante la búsqueda, lo que permite balancear precisión y velocidad.
*   se observa que aunque los documentos más relevantes coinciden, algunos resultados de menor ranking cambian. Además, la búsqueda con IVF_FLAT presenta menores tiempos de respuesta.

Debido a las limitaciones de Milvus Lite en entornos locales, no fue posible utilizar HNSW. Sin embargo, IVF_FLAT permite ilustrar adecuadamente el uso de índices ANN y la comparación entre precisión y velocidad.


## Parte 5 — Vector DB #3: Weaviate (búsqueda semántica con esquema)

### Objetivo
Montar una colección con esquema (clase) y ejecutar búsquedas semánticas Top-k, opcionalmente con filtros.

### Qué debes implementar
1. Conectar a Weaviate.
2. Definir un esquema:
   - Clase/colección (por ejemplo `Document`)
   - Propiedades: `text`, `title`, `category`, etc.
   - Vector asociado (embedding)
3. Insertar objetos con:
   - propiedades + vector
4. Consultar por similitud (Top-k) con `query_embedding`.
5. (Opcional) agregar un filtro por propiedad (metadata).

### Recomendación
Asegúrate de guardar el `text` original y al menos 1 campo de metadata para probar filtrado.

### Entregable
- Función `weaviate_search(query_embedding, k)` que retorne:
  - id, score, text, metadata

### Preguntas
- ¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?
- ¿Cómo describirías el trade-off de complejidad vs expresividad?


In [ ]:
pip install weaviate-client


In [ ]:
import weaviate
import time

try:
    client.close()
except:
    pass


client = weaviate.connect_to_embedded()


time.sleep(5)

if client.is_ready():
    print("conectado")
else:
    print("no")

INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 12265


conectado


In [ ]:
if client.collections.exists("Document"):
    client.collections.delete("Document")


In [ ]:
#crear colección
from weaviate.classes.config import Property, DataType, Configure

client.collections.create(
    name="Document",
    vectorizer_config=Configure.Vectorizer.none(),  # embeddings externos
    properties=[
        Property(name="text", data_type=DataType.TEXT),
    ],
)




In [ ]:
collection = client.collections.get("Document")


In [ ]:
with collection.batch.dynamic() as batch:
    for i in range(len(texts)):
        batch.add_object(
            properties={
                "text": texts[i]
            },
            vector=embeddings[i].tolist()
        )


In [ ]:
def weaviate_search(query_embedding, k=5):
    collection = client.collections.get("Document")

    response = collection.query.near_vector(
        near_vector=query_embedding[0].tolist(),
        limit=k,
        return_metadata=["distance"]
    )

    results = []
    for obj in response.objects:
        results.append({
            "score": 1 - obj.metadata.distance,
            "text": obj.properties["text"]
        })

    return results


In [ ]:
results = weaviate_search(query_vec, k=5)

for r in results:
    print(f"Score: {r['score']:.4f}")
    print(r["text"][:200])
    print("-" * 50)


Score: 0.8703
passage: Battery tester A battery tester is an electronic device intended for testing the state of an electric battery, going from a simple device for testing the charge actually present in the cells 
--------------------------------------------------
Score: 0.8618
passage: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of c
--------------------------------------------------
Score: 0.8401
passage: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-acid vehicle batteries. Their common principle is based on the empirical fact that after havi
--------------------------------------------------
Score: 0.8391
passage: ils. One was connected via a series resistor to the battery supply. The second was connected to the same battery supply via a second resistor and the resistor under test. The indi



*   En Weaviate v4, una Collection define un esquema semántico tipado, y cada objeto encapsula propiedades estructuradas junto a un vector. A diferencia de una tabla relacional, el acceso principal no es por claves o joins, sino por similitud semántica, permitiendo consultas híbridas que combinan significado y estructura.
*   Weaviate incrementa la complejidad inicial al requerir definición explícita de colecciones, tipos y relaciones, pero a cambio ofrece una expresividad superior para sistemas de recuperación semántica, filtrado contextual y búsquedas híbridas.

## Parte 6 — Vector Store #4: Chroma (prototipado rápido)

### Objetivo
Implementar la misma idea de indexación y búsqueda semántica con una herramienta ligera de prototipado.

### Qué debes implementar
1. Crear una colección.
2. Insertar:
   - ids
   - embeddings
   - documents (texto)
   - metadatas (opcional)
3. Consultar Top-k con `query_embedding`.

### Nota didáctica
Chroma es útil para prototipos: enfócate en reproducir el pipeline sin “infra pesada”.

### Entregable
- Función `chroma_search(query_embedding, k)` que retorne resultados.
- Una consulta con `k=5`.

### Preguntas
- ¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?
- ¿Qué limitaciones ves para un sistema en producción?


In [ ]:
pip install -U chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.5 MB/s eta 

In [ ]:
import chromadb
from chromadb.config import Settings

client = chromadb.Client(
    Settings(
        anonymized_telemetry=False
    )
)


In [ ]:
collection = client.create_collection(
    name="documents_chroma",
    metadata={"hnsw:space": "cosine"}
)


In [ ]:
BATCH_SIZE = 1000

ids = [str(i) for i in range(len(texts))]

for start in range(0, len(texts), BATCH_SIZE):
    end = start + BATCH_SIZE

    collection.add(
        ids=ids[start:end],
        embeddings=embeddings[start:end].tolist(),
        documents=texts[start:end]
    )

    print(f"Insertados {end if end < len(texts) else len(texts)} / {len(texts)}")


Insertados 1000 / 79104
Insertados 2000 / 79104
Insertados 3000 / 79104
Insertados 4000 / 79104
Insertados 5000 / 79104
Insertados 6000 / 79104
Insertados 7000 / 79104
Insertados 8000 / 79104
Insertados 9000 / 79104
Insertados 10000 / 79104
Insertados 11000 / 79104
Insertados 12000 / 79104
Insertados 13000 / 79104
Insertados 14000 / 79104
Insertados 15000 / 79104
Insertados 16000 / 79104
Insertados 17000 / 79104
Insertados 18000 / 79104
Insertados 19000 / 79104
Insertados 20000 / 79104
Insertados 21000 / 79104
Insertados 22000 / 79104
Insertados 23000 / 79104
Insertados 24000 / 79104
Insertados 25000 / 79104
Insertados 26000 / 79104
Insertados 27000 / 79104
Insertados 28000 / 79104
Insertados 29000 / 79104
Insertados 30000 / 79104
Insertados 31000 / 79104
Insertados 32000 / 79104
Insertados 33000 / 79104
Insertados 34000 / 79104
Insertados 35000 / 79104
Insertados 36000 / 79104
Insertados 37000 / 79104
Insertados 38000 / 79104
Insertados 39000 / 79104
Insertados 40000 / 79104
Insertado

In [ ]:
#busqueda top - k
def chroma_search(query_embedding, k=5):
    results = collection.query(
        query_embeddings=query_embedding.tolist(),
        n_results=k
    )

    output = []
    for i in range(len(results["ids"][0])):
        output.append({
            "id": results["ids"][0][i],
            "score": 1 - results["distances"][0][i],
            "text": results["documents"][0][i],
            "metadata": results["metadatas"][0][i] if results["metadatas"] else {}
        })

    return output


In [ ]:
#Consulta
results = chroma_search(query_vec, k=5)

for r in results:
    print(f"ID: {r['id']} | Score: {r['score']:.4f}")
    print(r["text"][:200])
    print("Metadata:", r["metadata"])
    print("-" * 50)


ID: 10176 | Score: 0.8703
passage: Battery tester A battery tester is an electronic device intended for testing the state of an electric battery, going from a simple device for testing the charge actually present in the cells 
Metadata: None
--------------------------------------------------
ID: 1 | Score: 0.8618
passage: Battery indicator A battery indicator (also known as a battery gauge) is a device which gives information about a battery. This will usually be a visual indication of the battery's state of c
Metadata: None
--------------------------------------------------
ID: 10177 | Score: 0.8401
passage: ing procedure, according to the type of battery being tested, such as the â€œ421â€ test for lead-acid vehicle batteries. Their common principle is based on the empirical fact that after havi
Metadata: None
--------------------------------------------------
ID: 37406 | Score: 0.8391
passage: ils. One was connected via a series resistor to the battery supply. The second was connect



*   Chroma fue significativamente más sencillo de implementar: no requiere levantar servicios, definir esquemas ni configurar índices explícitos. En pocas líneas se reproduce todo el pipeline de indexación y búsqueda
*   Carece de características clave para producción como escalabilidad distribuida, control fino de índices, tolerancia a fallos y seguridad avanzada. Además, su manejo de grandes volúmenes de datos y concurrencia es limitado.




## Parte 7 — SQL + vectores: PostgreSQL/pgvector (vector search transparente)

### Objetivo
Guardar embeddings en una tabla y ejecutar una consulta SQL de similitud.

### Qué debes implementar
1. Conectar a una base PostgreSQL con `pgvector` habilitado.
2. Crear una tabla (ej. `documents`) con:
   - `id` (PK)
   - `text` (texto)
   - `embedding` (vector(D))
   - metadata (columnas adicionales)
3. Insertar todos los documentos y embeddings.
4. Consultar Top-k por similitud, ordenando por distancia.

### Fórmula conceptual (lo que implementa tu SQL)
Para una consulta `q`, buscas:
$$ argmin_d \in D \; \text{dist}(\vec{q}, \vec{d})$$
donde `dist` puede ser L2 o una variante para cosine (según configuración).

### Entregable
- Función `pgvector_search(query_embedding, k)` que ejecute SQL y devuelva:
  - id, score/distancia, text, metadata

### Preguntas
- ¿Qué tan “explicable” te parece esta aproximación vs las otras?
- ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?
- ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?


In [ ]:
import psycopg2
import numpy as np


In [ ]:
conn = psycopg2.connect(
    host="localhost",
    port=5432,
    dbname="vectordb",
    user="postgres",
    password="postgres"
)

cur = conn.cursor()


OperationalError: connection to server at "localhost" (::1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
